In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os

# Paths
data_dir = 'animal'  # Update this to your folder path

# Data generators with splitting
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2  # 80-20 split for train-validation
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

# Load VGG16 model pre-trained on ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Build the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')  # Adjust output layer to match number of classes
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

# Plot training results
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

# Make predictions
import random

class_labels = list(train_generator.class_indices.keys())
for _ in range(5):  # Display 5 random predictions
    img, label = val_generator.next()
    true_label = np.argmax(label[0])
    prediction = model.predict(img)
    predicted_label = np.argmax(prediction[0])
    
    plt.imshow(img[0])
    plt.title(f"Predicted: {class_labels[predicted_label]}, Actual: {class_labels[true_label]}")
    plt.axis('off')
    plt.show()


Found 20995 images belonging to 10 classes.
Found 5244 images belonging to 10 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 22s 0us/step
Epoch 1/10


C:\Users\bonde\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


657/657 ━━━━━━━━━━━━━━━━━━━━ 4748s 7s/step - accuracy: 0.6797 - loss: 0.9598 - val_accuracy: 0.8501 - val_loss: 0.4731
Epoch 2/10
657/657 ━━━━━━━━━━━━━━━━━━━━ 4586s 7s/step - accuracy: 0.8953 - loss: 0.3284 - val_accuracy: 0.8627 - val_loss: 0.4250
Epoch 3/10
657/657 ━━━━━━━━━━━━━━━━━━━━ 4598s 7s/step - accuracy: 0.9357 - loss: 0.2145 - val_accuracy: 0.8736 - val_loss: 0.3922
Epoch 4/10
415/657 ━━━━━━━━━━━━━━━━━━━━ 22:48 6s/step - accuracy: 0.9692 - loss: 0.1301